Environment && Library 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"


import subprocess
with open(os.path.expanduser("~/successful NUMA node read from SysFS had negative value (-1)"), "r") as f:
    password = f.readline()
proc = subprocess.Popen(['sudo', '-S', 'sh', '-c', 'for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,universal_newlines= True)
stdout, stderr = proc.communicate(input=password+'\n')
print(stdout)
print(stderr)

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

[sudo] password for user: 


Configuration

In [2]:
augmentation = True
quant = True
exit = 1

Dataset

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
if(quant == True):
    x_train, x_test = x_train.reshape(-1, 28 ,28, 1).astype(np.uint8), x_test.reshape(-1, 28 ,28, 1).astype(np.uint8)
    validation_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 30, width_shift_range= 0.2, height_shift_range= 0.2, zoom_range= 0.2, data_format= 'channels_last', dtype= np.uint8)
elif(quant == False):
    x_train, x_test = x_train.reshape(-1, 28 ,28, 1).astype(np.float32) / 255.0, x_test.reshape(-1, 28 ,28, 1).astype(np.float32) / 255.0
    validation_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 30, width_shift_range= 0.2, height_shift_range= 0.2, zoom_range= 0.2, data_format= 'channels_last', dtype= np.float32)
validation_gen.fit(x_test)
validation_generator = validation_gen.flow(x_test, y_test, batch_size= 1, shuffle=True, seed= 2, save_to_dir= None)

Load Module

In [4]:
if(quant == True):
    trunk_1 = tf.lite.Interpreter(model_path= "./trunk_1_uint8.tflite")
    trunk_2 = tf.lite.Interpreter(model_path= "./trunk_2_uint8.tflite")
    trunk_3 = tf.lite.Interpreter(model_path= "./trunk_3_uint8.tflite")
    branch_1 = tf.lite.Interpreter(model_path= "./branch_1_uint8.tflite")
    branch_2 = tf.lite.Interpreter(model_path= "./branch_2_uint8.tflite")
    branch_3 = tf.lite.Interpreter(model_path= "./branch_3_uint8.tflite")
    branch_4 = tf.lite.Interpreter(model_path= "./branch_4_uint8.tflite")
    
elif(quant == False):
    trunk_1 = tf.lite.Interpreter(model_path= "./trunk_1_float32.tflite")
    trunk_2 = tf.lite.Interpreter(model_path= "./trunk_2_float32.tflite")
    trunk_3 = tf.lite.Interpreter(model_path= "./trunk_3_float32.tflite")
    branch_1 = tf.lite.Interpreter(model_path= "./branch_1_float32.tflite")
    branch_2 = tf.lite.Interpreter(model_path= "./branch_2_float32.tflite")
    branch_3 = tf.lite.Interpreter(model_path= "./branch_3_float32.tflite")
    branch_4 = tf.lite.Interpreter(model_path= "./branch_4_float32.tflite")

In [5]:
trunk_1.allocate_tensors()
trunk_2.allocate_tensors()
trunk_3.allocate_tensors()
branch_1.allocate_tensors()
branch_2.allocate_tensors()
branch_3.allocate_tensors()
branch_4.allocate_tensors()

Validation

In [6]:
predict_time, correct, epochs = 0, 0, 10000
accuracy_list = np.arange(1, epochs + 1).astype(np.float32)
latency_list = np.arange(1, epochs + 1).astype(np.float32)
if(epochs > 0):
    for i in range(epochs):
        x,y =validation_generator.next()
        start_time = time.time()
        for j in range(len(x)):
            input_details = trunk_1.get_input_details()
            output_details = trunk_1.get_output_details()
            trunk_1.set_tensor(input_details[0]['index'], x)
            trunk_1.invoke()
            output_data = trunk_1.get_tensor(output_details[0]['index'])
            if(exit == 1):
                input_details = branch_1.get_input_details()
                output_details = branch_1.get_output_details()
                branch_1.set_tensor(input_details[0]['index'], output_data)
                branch_1.invoke()
                output_data = branch_1.get_tensor(output_details[0]['index'])
            elif(exit > 1):
                input_details = trunk_2.get_input_details()
                output_details = trunk_2.get_output_details()
                trunk_2.set_tensor(input_details[0]['index'], output_data)
                trunk_2.invoke()
                output_data = trunk_2.get_tensor(output_details[0]['index'])

                if(exit == 2):
                    input_details = branch_2.get_input_details()
                    output_details = branch_2.get_output_details()
                    branch_2.set_tensor(input_details[0]['index'], output_data)
                    branch_2.invoke()
                    output_data = branch_2.get_tensor(output_details[0]['index'])
                elif(exit > 2):
                    input_details = trunk_3.get_input_details()
                    output_details = trunk_3.get_output_details()
                    trunk_3.set_tensor(input_details[0]['index'], output_data)
                    trunk_3.invoke()
                    output_data = trunk_3.get_tensor(output_details[0]['index'])

                    if(exit == 3):
                        input_details = branch_3.get_input_details()
                        output_details = branch_3.get_output_details()
                        branch_3.set_tensor(input_details[0]['index'], output_data)
                        branch_3.invoke()
                        output_data = branch_3.get_tensor(output_details[0]['index'])
                    elif(exit == 4):
                        input_details = branch_4.get_input_details()
                        output_details = branch_4.get_output_details()
                        branch_4.set_tensor(input_details[0]['index'], output_data)
                        branch_4.invoke()
                        output_data = branch_4.get_tensor(output_details[0]['index'])
                        
            index = np.argmax(output_data[0])
            predict_time += (time.time() - start_time)

            print(i, "\t", y[0], "\t", index, "\t", [round(k,3) for k in output_data[0]])
            if(index == y[0]):
                correct += 1
            if(epochs < 5):
                plt.imshow(x[0])
                plt.show()
            accuracy_list[i] = correct / (i + 1)
            latency_list[i] = predict_time * 1000 / (i + 1)

    print("predict_time:", (predict_time * 1000 / epochs), "ms")
    print("accuracy:", (correct * 100 / epochs), "%")

0 	 8 	 0 	 [91, 0, 0, 0, 0, 5, 0, 0, 91, 68]
1 	 8 	 8 	 [6, 0, 10, 0, 3, 0, 10, 0, 212, 14]
2 	 7 	 7 	 [22, 2, 76, 28, 0, 4, 3, 110, 3, 9]
3 	 0 	 0 	 [209, 4, 3, 4, 4, 2, 5, 8, 2, 15]
4 	 9 	 9 	 [0, 0, 0, 15, 2, 0, 0, 36, 1, 200]
5 	 8 	 8 	 [40, 0, 15, 9, 6, 1, 6, 0, 145, 35]
6 	 0 	 6 	 [43, 0, 0, 0, 32, 0, 171, 0, 7, 1]
7 	 2 	 2 	 [5, 0, 235, 2, 1, 0, 0, 0, 7, 7]
8 	 3 	 3 	 [2, 0, 58, 163, 0, 10, 3, 9, 9, 2]
9 	 0 	 0 	 [250, 0, 0, 5, 0, 0, 0, 0, 1, 0]
10 	 1 	 1 	 [11, 109, 2, 0, 16, 0, 0, 101, 0, 16]
11 	 0 	 4 	 [73, 25, 29, 2, 88, 0, 27, 1, 3, 7]
12 	 0 	 0 	 [231, 0, 2, 0, 0, 0, 0, 0, 0, 23]
13 	 8 	 8 	 [0, 1, 11, 0, 17, 0, 2, 6, 126, 94]
14 	 4 	 4 	 [0, 0, 11, 1, 206, 0, 0, 0, 26, 12]
15 	 1 	 1 	 [7, 122, 1, 2, 9, 1, 1, 87, 0, 27]
16 	 6 	 4 	 [11, 0, 0, 0, 118, 1, 118, 0, 7, 1]
17 	 2 	 2 	 [1, 0, 243, 10, 0, 0, 0, 0, 1, 1]
18 	 0 	 0 	 [126, 0, 0, 0, 3, 0, 126, 0, 0, 1]
19 	 1 	 1 	 [1, 184, 3, 1, 22, 1, 21, 22, 0, 0]
20 	 7 	 7 	 [1, 3, 14, 8, 1, 0, 0, 186, 0, 44]

Accuracy List

In [7]:
print("[", end= "")
for i in range(epochs):
    print(accuracy_list[i], end= "")
    if(i != epochs-1):
        print(",",end= " ")
print("]")

[0.0, 0.5, 0.6666667, 0.75, 0.8, 0.8333333, 0.71428573, 0.75, 0.7777778, 0.8, 0.8181818, 0.75, 0.7692308, 0.78571427, 0.8, 0.8125, 0.7647059, 0.7777778, 0.7894737, 0.8, 0.8095238, 0.8181818, 0.82608694, 0.8333333, 0.84, 0.84615386, 0.8518519, 0.85714287, 0.86206895, 0.8333333, 0.83870965, 0.8125, 0.8181818, 0.8235294, 0.82857144, 0.8055556, 0.8108108, 0.81578946, 0.82051283, 0.825, 0.8292683, 0.8333333, 0.8372093, 0.8181818, 0.82222223, 0.82608694, 0.80851066, 0.8125, 0.81632656, 0.82, 0.8235294, 0.8076923, 0.8113208, 0.7962963, 0.8, 0.8035714, 0.80701756, 0.79310346, 0.779661, 0.78333336, 0.7704918, 0.7580645, 0.7619048, 0.765625, 0.7692308, 0.77272725, 0.7761194, 0.7794118, 0.7826087, 0.78571427, 0.7887324, 0.7916667, 0.7808219, 0.7837838, 0.7866667, 0.7894737, 0.77922076, 0.78205127, 0.7848101, 0.7875, 0.79012346, 0.79268295, 0.78313255, 0.78571427, 0.7882353, 0.7790698, 0.77011496, 0.77272725, 0.76404494, 0.75555557, 0.74725276, 0.75, 0.75268817, 0.7553192, 0.75789475, 0.75, 0.7525

Latency List

In [8]:
print("[", end= "")
for i in range(epochs):
    print(latency_list[i], end= "")
    if(i != epochs-1):
        print(",",end= " ")
print("]")

[0.81944466, 0.4849434, 0.36557516, 0.29486418, 0.25134087, 0.22212665, 0.20125934, 0.18548965, 0.17314488, 0.16326904, 0.15590407, 0.14901161, 0.14308783, 0.13806139, 0.13370514, 0.12984872, 0.12647404, 0.12347433, 0.12080293, 0.118386745, 0.11617797, 0.115784734, 0.11525983, 0.11475881, 0.114564896, 0.113744006, 0.11414069, 0.11288268, 0.11153879, 0.11027654, 0.10906497, 0.10800362, 0.10699214, 0.106033154, 0.10510172, 0.10534128, 0.104588434, 0.10377482, 0.1029846, 0.10223389, 0.10154887, 0.10089647, 0.10039086, 0.099789016, 0.099224515, 0.09866383, 0.09814729, 0.097975135, 0.09802409, 0.0980711, 0.09844817, 0.09909043, 0.09888523, 0.09850661, 0.09808107, 0.09766221, 0.09788965, 0.09801059, 0.09762634, 0.09723902, 0.09685657, 0.096486464, 0.09615081, 0.09583309, 0.09552515, 0.09521932, 0.094979554, 0.09469776, 0.094417214, 0.09415831, 0.09389998, 0.09365545, 0.093407825, 0.09322488, 0.092995964, 0.093265586, 0.09360871, 0.09388496, 0.09415422, 0.09446442, 0.09486999, 0.09524822, 0.0